In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Tmall_challenge')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install deepctr
!pip install tqdm

In [0]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from deepctr.inputs import SparseFeat, VarLenSparseFeat, DenseFeat
from deepctr.models import DIN
from tqdm import tqdm_notebook

In [4]:
# 加载样本
'''train_data = pd.read_csv('./sample/train_data_sample.csv')
test_data = pd.read_csv('./sample/test_data_sample.csv')
user_info = pd.read_csv('./sample/user_info_sample.csv')
user_log = pd.read_csv('./sample/user_log_sample.csv')
train_data1 = pd.read_csv('./train_format2.csv')'''

"train_data = pd.read_csv('./sample/train_data_sample.csv')\ntest_data = pd.read_csv('./sample/test_data_sample.csv')\nuser_info = pd.read_csv('./sample/user_info_sample.csv')\nuser_log = pd.read_csv('./sample/user_log_sample.csv')\ntrain_data1 = pd.read_csv('./train_format2.csv')"

In [0]:
# 全量数据

train_data = pd.read_csv('./train_format1.csv')
test_data = pd.read_csv('./test_format1.csv')
user_info = pd.read_csv('./user_info_format1.csv')
user_log = pd.read_csv('./user_log_format1.csv')
train_data1 = pd.read_csv('./train_format2.csv')

In [6]:
train_data['origin'] = 'train'
test_data['origin'] = 'test'
matrix = pd.concat([train_data, test_data], ignore_index=True, sort=False)
# 数据处理
user_log.rename(columns={'seller_id': 'merchant_id'}, inplace=True)
user_log['brand_id'].fillna(0, inplace=True)
user_log['brand_id'] = user_log['brand_id'].astype('int64')
user_log['time_stamp'] = pd.to_datetime(user_log['time_stamp'], format='%H%M')
user_info['age_range'].fillna(0, inplace=True)
user_info = user_info.replace([8], [7])
user_info['age_range'] = user_info['age_range'].astype('int64')
user_info['gender'].fillna(2, inplace=True)
user_info['gender'] = user_info['gender'].astype('int64')

'''# 对离散特征做LabelEncoder
lbe_merchant_id=LabelEncoder()
lbe_merchant_id.fit(np.r_[0,user_log['merchant_id'].values])
user_log['merchant_id']=lbe_merchant_id.transform(user_log['merchant_id'])
matrix['merchant_id']=lbe_merchant_id.transform(matrix['merchant_id'])

lbe_user_id=LabelEncoder()
user_log['user_id']=lbe_user_id.fit_transform(user_log['user_id'])
user_info['user_id']=lbe_user_id.transform(user_info['user_id'])
matrix['user_id']=lbe_user_id.transform(matrix['user_id'])

lbe_item_id=LabelEncoder()
user_log['item_id']=lbe_item_id.fit_transform(user_log['item_id'])
lbe_cat_id=LabelEncoder()
user_log['cat_id']=lbe_cat_id.fit_transform(user_log['cat_id'])
lbe_brand_id=LabelEncoder()
user_log['brand_id']=lbe_brand_id.fit_transform(user_log['brand_id'])'''

"# 对离散特征做LabelEncoder\nlbe_merchant_id=LabelEncoder()\nlbe_merchant_id.fit(np.r_[0,user_log['merchant_id'].values])\nuser_log['merchant_id']=lbe_merchant_id.transform(user_log['merchant_id'])\nmatrix['merchant_id']=lbe_merchant_id.transform(matrix['merchant_id'])\n\nlbe_user_id=LabelEncoder()\nuser_log['user_id']=lbe_user_id.fit_transform(user_log['user_id'])\nuser_info['user_id']=lbe_user_id.transform(user_info['user_id'])\nmatrix['user_id']=lbe_user_id.transform(matrix['user_id'])\n\nlbe_item_id=LabelEncoder()\nuser_log['item_id']=lbe_item_id.fit_transform(user_log['item_id'])\nlbe_cat_id=LabelEncoder()\nuser_log['cat_id']=lbe_cat_id.fit_transform(user_log['cat_id'])\nlbe_brand_id=LabelEncoder()\nuser_log['brand_id']=lbe_brand_id.fit_transform(user_log['brand_id'])"

In [0]:
matrix.drop('prob', axis=1, inplace=True)
matrix = matrix.merge(user_info, on='user_id', how='left')


In [8]:
# 用户特征处理
groups = user_log.groupby(['user_id'])
temp = groups.size().reset_index().rename(columns={0: 'u1'})
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['item_id'].agg([('u2', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['cat_id'].agg([('u3', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['merchant_id'].agg([('u4', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['brand_id'].agg([('u5', 'nunique')]).reset_index()
matrix = matrix.merge(temp, on='user_id', how='left')
temp = groups['time_stamp'].agg([('F_time', 'min'), ('L_time', 'max')]).reset_index()
temp['u6'] = (temp['L_time'] - temp['F_time']).dt.seconds/3600
matrix = matrix.merge(temp[['user_id', 'u6']], on='user_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'u7', 1: 'u8', 2: 'u9', 3: 'u10'})
matrix = matrix.merge(temp, on='user_id', how='left')

# 商户特征处理
groups = user_log.groupby(['merchant_id'])
temp = groups.size().reset_index().rename(columns={0: 'm1'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['user_id', 'item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'user_id': 'm2', 'item_id': 'm3', 'cat_id': 'm4', 'brand_id': 'm5'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'm6', 1: 'm7', 2: 'm8', 3: 'm9'})
matrix = matrix.merge(temp, on='merchant_id', how='left')
temp = train_data1[train_data1['label'] == -1].groupby('merchant_id').size().reset_index().rename(columns={0: 'm10'})
matrix = matrix.merge(temp, on='merchant_id', how='left')

import gc
del user_info, train_data1
gc.collect()

0

In [0]:
groups = user_log.groupby(['user_id', 'merchant_id'])
temp = groups.size().reset_index().rename(columns={0: 'um1'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['item_id', 'cat_id', 'brand_id'].nunique().reset_index().rename(columns={'item_id': 'um2', 'cat_id': 'um3', 'brand_id': 'um4'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['action_type'].value_counts().unstack().reset_index().rename(columns={0: 'um5', 1: 'um6', 2: 'um7', 3: 'um8'})
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')
temp = groups['time_stamp'].agg([('first', 'min'), ('last', 'max')]).reset_index()
temp['um9'] = (temp['last'] - temp['first']).dt.seconds/3600
temp.drop(['last', 'first'], axis=1, inplace=True)
matrix = matrix.merge(temp, on=['user_id', 'merchant_id'], how='left')

In [0]:
matrix['r1'] = matrix['u9'] / matrix['u7']
matrix['r2'] = matrix['m8'] / matrix['m6']
matrix['r3'] = matrix['um7'] / matrix['um5']
matrix.fillna(0, inplace=True)
temp = pd.get_dummies(matrix['age_range'], prefix='age')
matrix = pd.concat([matrix, temp], axis=1)
temp = pd.get_dummies(matrix['gender'], prefix='gen')
matrix = pd.concat([matrix, temp], axis=1)
matrix.drop(['age_range', 'gender'], axis=1, inplace=True)

In [0]:
lbe_action_type = {0:1, 1:2, 2:3, 3:4}
user_log['action_type'] = user_log['action_type'].map(lbe_action_type)
temp = pd.DataFrame(user_log.groupby('user_id')['merchant_id','action_type'].agg(lambda x:list(x)))
temp.columns = ['hist_merchant_id','hist_action_type']
matrix = matrix.merge(temp, on='user_id', how='left')

max_len = 500
for feature in ['hist_merchant_id', 'hist_action_type']:
    matrix[feature] = matrix[feature].map(lambda x: np.array(x + [0] * (max_len - len(x)))[:max_len])


In [0]:
train_data = matrix[matrix['origin'] == 'train'].drop('origin', axis=1)
test = matrix[matrix['origin'] == 'test'].drop(['label', 'origin'], axis=1)
x_train, y_train = train_data.drop('label', axis=1), train_data['label']

In [0]:
x_train['action_type']=3
feature_columns = []
for col in x_train.columns:
    if col not in ['hist_merchant_id', 'hist_action_type']:
        num = x_train[col].nunique()
        if num > 10000:
            dim = 10
        else:
            if num > 1000:
                dim = 8
            else:
                dim = 4
        if col in ['user_id', 'merchant_id','age_0', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7', 'gen_0', 'gen_1', 'gen_2']:
            feature_columns += [SparseFeat(col, x_train[col].max()+1, embedding_dim=dim)]
        elif col  == 'action_type':
            feature_columns += [SparseFeat(col, 4+1, embedding_dim=dim)]
        else:
            feature_columns += [DenseFeat(col, 1)]

feature_columns += [VarLenSparseFeat(SparseFeat('hist_merchant_id', vocabulary_size=user_log['merchant_id'].nunique()+1, embedding_dim=8, embedding_name='merchant_id'), maxlen=max_len),
                    VarLenSparseFeat(SparseFeat('hist_action_type', vocabulary_size=user_log['action_type'].nunique()+1, embedding_dim=4, embedding_name='action_type'), maxlen=max_len)]
hist_features = ['merchant_id', 'action_type']

In [14]:
feature_columns

[SparseFeat(name='user_id', vocabulary_size=424171, embedding_dim=10, use_hash=False, dtype='int32', embedding_name='user_id', group_name='default_group'),
 SparseFeat(name='merchant_id', vocabulary_size=4994, embedding_dim=8, use_hash=False, dtype='int32', embedding_name='merchant_id', group_name='default_group'),
 DenseFeat(name='u1', dimension=1, dtype='float32'),
 DenseFeat(name='u2', dimension=1, dtype='float32'),
 DenseFeat(name='u3', dimension=1, dtype='float32'),
 DenseFeat(name='u4', dimension=1, dtype='float32'),
 DenseFeat(name='u5', dimension=1, dtype='float32'),
 DenseFeat(name='u6', dimension=1, dtype='float32'),
 DenseFeat(name='u7', dimension=1, dtype='float32'),
 DenseFeat(name='u8', dimension=1, dtype='float32'),
 DenseFeat(name='u9', dimension=1, dtype='float32'),
 DenseFeat(name='u10', dimension=1, dtype='float32'),
 DenseFeat(name='m1', dimension=1, dtype='float32'),
 DenseFeat(name='m2', dimension=1, dtype='float32'),
 DenseFeat(name='m3', dimension=1, dtype='floa

In [15]:
model = DIN(feature_columns, hist_features)
model.compile('adam', 'binary_crossentropy', metrics=['binary_crossentropy'])

feature_names = list(x_train.columns)
train_model_input = {name: x_train[name].values for name in feature_names}
for feature in ['hist_merchant_id', 'hist_action_type']:
    l = []
    for i in tqdm_notebook(train_model_input[feature]):
        l.append(i)
    train_model_input[feature] = np.array(l)
history = model.fit(train_model_input, y_train.map(float), verbose=True, epochs=10, validation_split=0.2, batch_size=512)

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'attention_sequence_pooling_layer/local_activation_unit/kernel:0' shape=(40, 1) dtype=float32>
  <tf.Variable 'attention_sequence_pooling_layer/local_activation_unit/bias:0' shape=(1,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Epoch 1/10


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


408/408 [==============================] - 127s 312ms/step - loss: 1.0672 - binary_crossentropy: 1.0674 - val_loss: 0.9430 - val_binary_crossentropy: 0.9431
Epoch 2/10
408/408 [==============================] - 129s 317ms/step - loss: 0.9436 - binary_crossentropy: 0.9433 - val_loss: 0.9430 - val_binary_crossentropy: 0.9431
Epoch 3/10
408/408 [==============================] - 129s 316ms/step - loss: 0.9433 - binary_crossentropy: 0.9433 - val_loss: 0.9430 - val_binary_crossentropy: 0.9431
Epoch 4/10
408/408 [==============================] - 130s 319ms/step - loss: 0.9431 - binary_crossentropy: 0.9433 - val_loss: 0.9430 - val_binary_crossentropy: 0.9431
Epoch 5/10
408/408 [==============================] - 129s 317ms/step - loss: 0.9435 - binary_crossentropy: 0.9433 - val_loss: 0.9430 - val_binary_crossentropy: 0.9431
Epoch 6/10
408/408 [==============================] - 129s 317ms/step - loss: 0.9434 - binary_crossentropy: 0.9433 - val_loss: 0.9430 - val_binary_crossentropy: 0.9431
Epo

In [16]:
test['action_type']=3
test_model_input = {name:test[name].values for name in feature_names}

for fea in ['hist_merchant_id','hist_action_type']:
    l = []
    for i in tqdm_notebook(test_model_input[fea]):
        l.append(i)
    test_model_input[fea]=np.array(l)

# 得到预测结果
prob = model.predict(test_model_input)
test_data['prob'] = prob

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [0]:
model.predict?